# Utilisation de la caméra USB et CSI avec Jupyter

Dans ce notebook, nous allons étudier comment utiliser la caméra USB et CSI avec Jupyter.

<img src="https://github.com/AlexandreBourrieau/JetsonNano/blob/main/images/usbcam_setup_sm.jpg?raw=true" width=600>

### Recherche de la caméra

In [ ]:
%config IPCompleter.greedy=True
import cv2
from matplotlib import pyplot as plt
import os

On regarde les caméras branchées :

In [ ]:
!ls -ltrh /dev/video*

### Définition de la classe Camera

In [ ]:
import traitlets
import threading
import atexit
import numpy as np


class Camera(traitlets.HasTraits):
    type_camera = traitlets.Unicode("CSI")
    capture_device = traitlets.Integer(default_value=0)
    capture_width = traitlets.Integer(default_value=1280)
    capture_height = traitlets.Integer(default_value=720)
    display_width = traitlets.Integer(default_value=640)
    display_height = traitlets.Integer(default_value=480)
    fps = traitlets.Integer(default_value=30)
    flip = traitlets.Integer(default_value=0)
    image = traitlets.Any()
    video_on = traitlets.Bool(default_value=False)
    
    def __init__(self,*args,**kwargs):
        super(Camera, self).__init__(*args, **kwargs)
        self._running = False
        
        if self.type_camera.find("CSI")>=0:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_CSI(),cv2.CAP_GSTREAMER)
        else:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_USB(),cv2.CAP_GSTREAMER)

        if self.cap.isOpened():
            print("Caméra initialisée")
        else:
            print("Erreur d'ouverture du flux vidéo")
        atexit.register(self.cap.release)
    
    # Lecture d'une frame
    def capture_image(self):
        re, image = self.cap.read()
        if re:
            image_resized = cv2.resize(image,(int(self.display_width),int(self.display_height)))
        return image_resized
    
    # ON/OFF de la capture vidéo
    def capture_video(self,run=False):
        if run is True:
            self.video_on = True
        else:
            self.video_on = False
    
    # Lecture d'un flux vidéo
    def _capture_video(self):
        while True:
            if not self._running:
                break
            self.image = self.capture_image()

            
    # Détachement de la caméra
    def release(self):
        self.cap.release()

    # Définition du pipeline pour la caméra CSI
    def _gstreamer_pipeline_CSI(self):
        return("nvarguscamerasrc sensor-id=%d ! "
                "video/x-raw(memory:NVMM),"
                "width=(int)%d,height=(int)%d,"
                "format=(string)NV12, framerate=(fraction)%d/1 ! "
                "nvvidconv flip-method=%d ! "
                "video/x-raw,"
                "width=(int)%d,height=(int)%d,"
                "format=(string)BGRx ! videoconvert ! "
                "video/x-raw, format=(string)BGR ! "
                "appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip, self.display_width,self.display_height))

    # Définition du pipeline pour la USB
    def _gstreamer_pipeline_USB(self):
        return("v4l2src device=/dev/video%d ! "
               "video/x-raw, width=(int)%d, height=(int)%d, framerate=(fraction)%d/1 ! "
               "videoflip method=%d ! "
               "videoconvert ! "
               "video/x-raw, format=(string)BGR ! appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip))
    
    # Surveillance de la variable "video_on"
    @traitlets.observe('video_on')
    def _on_running(self, change):
        if change['new'] and not change['old']:
            # not running -> running
            self._running = True
            self.thread = threading.Thread(target=self._capture_video)
            self.thread.start()
        elif change['old'] and not change['new']:
            # running -> not running
            self._running = False
            self.thread.join()

### Instanciation de la classe Camera

On commence par instancier un objet `camera` de la classe Camera en passant les arguments suivants :
 - type_camera : USB  
 - capture_device : Port sur lequel est branché la caméra USB  
 - capture_width et capture_height : Résolution de la capture  
 - display_width et display_height : Résolution de la vidéo ou de l'image retournée  
 - fps : Framerate  
 - flip : Retournement

In [ ]:
camera = Camera(type_camera="CSI",capture_device=0,
                capture_width=640,capture_height=480,
                display_width=320,display_height=200,
                fps=30,flip=2)

On peut ensuite capturer une image avec la caméra en appelant la méthode `capture_image`. On peut observer ainsi le format du tenseur retourné :

In [ ]:
image = camera.capture_image()
print(image.shape)

Pour afficher l'image on peut par exemple utiliser la méthode `imgshow` de pyplot :

In [ ]:
plt.imshow(image)

### Création d'un widget pour visualiser l'image

Au lieu d'utiliser pyplot, on peut créer un Widget pour afficher l'image. Il faut cependant la convertir au format jpeg avant de l'afficher à l'aide de la fonction définie ci-dessous :

In [ ]:
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
import ipywidgets
from IPython.display import display
from ipywidgets import Layout

image = camera.capture_image()
image_widget = ipywidgets.Image(format='jpeg',layout=Layout(width="320px", height="200px"))
image_widget.value = bgr8_to_jpeg(image)

display(image_widget)

### Visualiser le flux vidéo de la caméra

Pour visualiser le flux vidéo, il faut appeler la méthode `capture_video` de la classe camera.

In [ ]:
camera.capture_video(run=True)

def update_image(change):
    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)

camera.observe(update_image,names='image')

In [ ]:
camera.unobserve(update_image,names='image')
camera.capture_video(run=False)

In [ ]:
camera.capture_video(run=False)

### Autre méthode pour visualiser le flux vidéo de la caméra

In [ ]:
camera.capture_video(run=True)
camera_link = traitlets.dlink((camera, 'image'), (image_widget, 'value'), transform=bgr8_to_jpeg)

In [ ]:
camera_link.unlink()
camera.capture_video(run=False)

# Fermeture de la caméra

Avant de quitter le  notebook, il faut fermer la caméra :

In [ ]:
camera.release()
del camera

In [ ]:
os._exit(00)